In [1]:
# hint for next try:
# keep track of deleted root edges.
# if at any point a new edge to be deleted exists in the already deleted edges, the just break out from the while loop

In [2]:
import sympy as sp
import minterpy as mp
import numpy as np
from minterpy.pointcloud_utils import *

from mpl_toolkits import mplot3d
%matplotlib inline

from mpl_toolkits.mplot3d import axes3d

import torch
import torchvision
from torchvision import transforms, datasets

import random
import numpy as np
import matplotlib.pyplot as plt

import os
from operator import itemgetter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import ot

from sklearn.neighbors import NearestNeighbors

import ripser
import persim
from persim import plot_diagrams

from operator import itemgetter


#do pip installs as follows to use vedo for plotting point clouds
#pip install vedo
#pip install ipyvtklink

import numpy as np
from vedo import *

import matplotlib.pyplot
from mpl_toolkits.mplot3d import Axes3D

import time
torch.set_printoptions(precision=8)


In [3]:
# function to check whether the selected edge is going to close a potential loop

def get_all_loops(actual_new_test, my_edge):

    left_ind = my_edge[0][0]
    right_ind = my_edge[0][1]
    found_right_ind = False
    going_nowhere= False

    new_test = actual_new_test

    tracker = 0
    no_branches_formed = True
    while (not(found_right_ind) or not(going_nowhere)):

        positions1 = (new_test == left_ind).nonzero(as_tuple=False)
        print("adjaoined point index ",positions1)
        print("Shape of positions found", positions1.shape)

        print("Number of branches rising : ", positions1.shape[0])

        if(positions1.shape[0]>1):
            edge_to_delete = new_test[positions1[0][0]]
            no_branches_formed = False
            print("The edge to be deleted ", edge_to_delete)
            print("The shape of the actual edge to be deleted ", edge_to_delete.shape[0])
        
        branches_rising = positions1.shape[0]
        print("To fix for loop range",branches_rising)

        if(positions1.shape[0]==0):
            going_nowhere= True
            print("There is no loop")
            print("I got nowhere")
            if(no_branches_formed):
                break
            
            print("I should delete the edge : ", edge_to_delete)
            left_ind = my_edge[0][0]

            deletable_edge_position1 = (actual_new_test == edge_to_delete[0]).nonzero(as_tuple=False)
            deletable_edge_position2 = (actual_new_test == edge_to_delete[1]).nonzero(as_tuple=False)

            deletable_edge_position1 = deletable_edge_position1[:,0]

            deletable_edge_position2 = deletable_edge_position2[:,0]

            print("What happened : ", deletable_edge_position1, deletable_edge_position1)
            
            a_cat_b1, counts1 = torch.cat([deletable_edge_position1, deletable_edge_position2]).unique(return_counts=True)
            deletable_row_position = a_cat_b1[torch.where(counts1.gt(1))]
            print("Here it is : ", deletable_row_position.shape[0])

            if(deletable_row_position.shape[0]==0):
                print(" Your edge is not going to form a cycle ")
                going_nowhere = True
                break

            deletable_row_position = deletable_row_position[0]
            
            print("Position of edge to be deleted as columns ", deletable_edge_position1, deletable_edge_position2)
            print("Deletable row position : ", deletable_row_position)

            actual_new_test = torch.cat((actual_new_test[:deletable_row_position], actual_new_test[deletable_row_position+1:]))
            new_test = actual_new_test
            print("shape of edge to delete", edge_to_delete.shape[0])
            print(new_test)
            positions1 = (new_test == left_ind).nonzero(as_tuple=False)

            if(tracker ==0):
                break

        if(positions1.shape[0]>1):
            edge_to_delete = new_test[positions1[0][0]]
            no_branches_formed = False
                
        first_position = positions1[0][0]
        print("Where is it ",first_position)
        adj_edge1 = new_test[positions1[0][0]]
        print("The new edge is ",adj_edge1)
        
        

        other_end1 = abs(positions1 - torch.tensor([[0, 1]]))
        print("The new point found is at the index ",other_end1)


        consec_pt1 = new_test[other_end1[0][0]][other_end1[0][1]]
        consec_pt1 = int(consec_pt1)
        print(" The new point is ",consec_pt1)

        if(consec_pt1 == right_ind):
            found_right_ind = True
            print("This forms a loop")
            print("traversions : ", tracker)
            break

        else:
            left_ind = consec_pt1
            new_test = torch.cat((new_test[:first_position], new_test[first_position+1:]))
            tracker = tracker+1

    print("The cycle exists : ", found_right_ind)

    return found_right_ind



In [4]:
test_edge_ = torch.tensor([ 13, 5])

def check_edge_presence(test_edge_, set_of_edges):    
    
    test_edge_position1 = (set_of_edges == test_edge_[0]).nonzero(as_tuple=False)
    test_edge_position2 = (set_of_edges == test_edge_[1]).nonzero(as_tuple=False)
    test_edge_position1 = test_edge_position1[:,0]
    test_edge_position2 = test_edge_position2[:,0]
    a_cat_b1, counts1 = torch.cat([test_edge_position1, test_edge_position2]).unique(return_counts=True)
    test_edge_position = a_cat_b1[torch.where(counts1.gt(1))]
    
    return not(test_edge_position.shape[0]==0)


In [17]:
# function to check whether the selected edge is going to close a potential loop

def get_all_loops(actual_new_test, my_edge):
    
    num_loops_found = 0
    next_loop_test = actual_new_test
    left_ind = my_edge[0][0]
    right_ind = my_edge[0][1]
    found_right_ind = False
    going_nowhere= False

    new_test = actual_new_test
    tracker = 0
    no_branches_formed = True
    
    origin_root_tracker = actual_new_test
    original_roots = (origin_root_tracker == left_ind).nonzero(as_tuple=False)
    root_extenders_positions = abs(original_roots - torch.tensor([[0, 1]]))
    left_positions = root_extenders_positions[:,0]
    right_positions = root_extenders_positions[:,1]
    root_origin_edge_set = origin_root_tracker[left_positions]
    print("Root origin edge sets ", root_origin_edge_set)
    
    print("Now check if certain edge is in root_origin_edge_set ")
    '''test_edge_ = torch.tensor([ 13, 5])
    test_edge_position1 = (root_origin_edge_set == test_edge_[0]).nonzero(as_tuple=False)
    test_edge_position2 = (root_origin_edge_set == test_edge_[1]).nonzero(as_tuple=False)
    test_edge_position1 = test_edge_position1[:,0]
    test_edge_position2 = test_edge_position2[:,0]
    a_cat_b1, counts1 = torch.cat([test_edge_position1, test_edge_position2]).unique(return_counts=True)
    test_edge_position = a_cat_b1[torch.where(counts1.gt(1))]
    
    print("check what is this ",test_edge_position.shape[0])'''
    
    presence_check = check_edge_presence(torch.tensor([ 10, 5]), root_origin_edge_set)
    print(presence_check)

    while (not(found_right_ind) or not(going_nowhere)):

        positions1 = (new_test == left_ind).nonzero(as_tuple=False)
        print("adjaoined point index ",positions1)
        print("Shape of positions found", positions1.shape)

        print("Number of branches rising : ", positions1.shape[0])

        if(positions1.shape[0]>1):
            edge_to_delete = new_test[positions1[0][0]]
            no_branches_formed = False
            print("The edge to be deleted ", edge_to_delete)
            print("The shape of the actual edge to be deleted ", edge_to_delete.shape[0])
        
        branches_rising = positions1.shape[0]
        print("To fix for loop range",branches_rising)

        if(positions1.shape[0]==0):
            going_nowhere= True
            print("There is no loop")
            print("I got nowhere")
            if(no_branches_formed):
                break
            
            print("I should delete the edge : ", edge_to_delete)
            left_ind = my_edge[0][0]
            print('edge_to_delete',edge_to_delete)
            print()
            print()
            
            if(check_edge_presence(edge_to_delete, root_origin_edge_set)):
                actual_new_test = next_loop_test
            
            deletable_edge_position1 = (actual_new_test == edge_to_delete[0]).nonzero(as_tuple=False)
            
            deletable_edge_position2 = (actual_new_test == edge_to_delete[1]).nonzero(as_tuple=False)
            
            deletable_edge_position1 = deletable_edge_position1[:,0]

            deletable_edge_position2 = deletable_edge_position2[:,0]

            print("What happened : ", deletable_edge_position1, deletable_edge_position1)
            
            a_cat_b1, counts1 = torch.cat([deletable_edge_position1, deletable_edge_position2]).unique(return_counts=True)
            deletable_row_position = a_cat_b1[torch.where(counts1.gt(1))]
            print("deletable_row_position ",deletable_row_position)
            print("Here it is : ", deletable_row_position.shape[0])

            if(deletable_row_position.shape[0]==0):
                print(" Your edge is not going to form a cycle ")
                going_nowhere = True
                break

            deletable_row_position = deletable_row_position[0]
            
            print("Position of edge to be deleted as columns ", deletable_edge_position1, deletable_edge_position2)
            print("Deletable row position : ", deletable_row_position)

            actual_new_test = torch.cat((actual_new_test[:deletable_row_position], actual_new_test[deletable_row_position+1:]))
            new_test = actual_new_test
            print("shape of edge to delete", edge_to_delete.shape[0])
            print(new_test)
            positions1 = (new_test == left_ind).nonzero(as_tuple=False)
            print("this shape of position1 ", positions1.shape)
            print("the actual position1 is ", positions1)

            if(tracker ==0):
                break

        if(positions1.shape[0]>1):
            edge_to_delete = new_test[positions1[0][0]]
            print("I should also delete this edge ; ", edge_to_delete)
            no_branches_formed = False
                
        first_position = positions1[0][0]
        print("Where is it ",first_position)
        adj_edge1 = new_test[positions1[0][0]]
        print("go")
        print()
        print("The new edge is/ is the only ",adj_edge1)
        #This is where you have to go on concatenating the edges of the loop
        print()
        print("Ye")

        other_end1 = abs(positions1 - torch.tensor([[0, 1]]))
        print("The new point found is at the index ",other_end1)


        consec_pt1 = new_test[other_end1[0][0]][other_end1[0][1]]
        consec_pt1 = int(consec_pt1)
        print(" The new point is ",consec_pt1)

        if(consec_pt1 == right_ind):
            found_right_ind = True
            print("This forms a loop")
            print("traversions : ", tracker)
            print("But what was the edge to delete ? ", edge_to_delete)
            #actual_new_test = next_loop_test
            deletable_edge_position1 = (next_loop_test == edge_to_delete[0]).nonzero(as_tuple=False)
            deletable_edge_position2 = (next_loop_test == edge_to_delete[1]).nonzero(as_tuple=False)
            deletable_edge_position1 = deletable_edge_position1[:,0]
            deletable_edge_position2 = deletable_edge_position2[:,0]
            a_cat_b1, counts1 = torch.cat([deletable_edge_position1, deletable_edge_position2]).unique(return_counts=True)
            deletable_row_position = a_cat_b1[torch.where(counts1.gt(1))]
            deletable_row_position = deletable_row_position[0]
            new_test = torch.cat((next_loop_test[:deletable_row_position], next_loop_test[deletable_row_position+1:]))
            
            if(check_edge_presence(edge_to_delete, root_origin_edge_set)):
                next_loop_test = new_test
            actual_new_test = new_test
            left_ind = my_edge[0][0]
            print("lets see")
            print(new_test)
            found_right_ind = False
            going_nowhere= False
            no_branches_formed = True
            tracker = 0
            num_loops_found = num_loops_found + 1

            #break

        else:
            left_ind = consec_pt1
            new_test = torch.cat((new_test[:first_position], new_test[first_position+1:]))
            tracker = tracker+1

    print("The cycle exists : ", found_right_ind)
    print("num loops found ", num_loops_found)
    return found_right_ind



In [18]:
#testing the function
current_edge_collec = torch.tensor([[10., 11.],
        [4., 15.],
        [14., 15.],
        [11., 12.],
        [2., 3.],
        [ 5., 14.],
        [11., 9.],
        [ 12., 13.],
        [7., 8.],
        [ 3., 4.],
        [9., 2.],
        [13., 5.],
        [8., 4.],
        [ 1.,  2.],
        [1., 5.]])
#my_edge = torch.tensor([[ 9, 18]])

curr_edge = torch.tensor([[ 5, 7]])


In [19]:
get_all_loops(current_edge_collec, curr_edge)

Root origin edge sets  tensor([[ 5., 14.],
        [13.,  5.],
        [ 1.,  5.]])
Now check if certain edge is in root_origin_edge_set 
False
adjaoined point index  tensor([[ 5,  0],
        [11,  1],
        [14,  1]])
Shape of positions found torch.Size([3, 2])
Number of branches rising :  3
The edge to be deleted  tensor([ 5., 14.])
The shape of the actual edge to be deleted  2
To fix for loop range 3
I should also delete this edge ;  tensor([ 5., 14.])
Where is it  tensor(5)
go

The new edge is/ is the only  tensor([ 5., 14.])

Ye
The new point found is at the index  tensor([[ 5,  1],
        [11,  0],
        [14,  0]])
 The new point is  14
adjaoined point index  tensor([[2, 0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(2)
go

The new edge is/ is the only  tensor([14., 15.])

Ye
The new point found is at the index  tensor([[2, 1]])
 The new point is  15
adjaoined point index  tensor([[1, 1]])
Shape of

adjaoined point index  tensor([[5, 1]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(5)
go

The new edge is/ is the only  tensor([1., 2.])

Ye
The new point found is at the index  tensor([[5, 0]])
 The new point is  1
adjaoined point index  tensor([[5, 0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(5)
go

The new edge is/ is the only  tensor([1., 5.])

Ye
The new point found is at the index  tensor([[5, 1]])
 The new point is  5
adjaoined point index  tensor([], size=(0, 2), dtype=torch.int64)
Shape of positions found torch.Size([0, 2])
Number of branches rising :  0
To fix for loop range 0
There is no loop
I got nowhere
I should delete the edge :  tensor([13.,  5.])
edge_to_delete tensor([13.,  5.])


What happened :  tensor([ 6, 10]) tensor([ 6, 10])
deletable_row_position  tensor([10])
Here it is :  1
Position of edge to be deleted as

To fix for loop range 2
I should also delete this edge ;  tensor([10., 11.])
Where is it  tensor(0)
go

The new edge is/ is the only  tensor([10., 11.])

Ye
The new point found is at the index  tensor([[0, 0],
        [3, 1]])
 The new point is  10
adjaoined point index  tensor([], size=(0, 2), dtype=torch.int64)
Shape of positions found torch.Size([0, 2])
Number of branches rising :  0
To fix for loop range 0
There is no loop
I got nowhere
I should delete the edge :  tensor([10., 11.])
edge_to_delete tensor([10., 11.])


What happened :  tensor([0]) tensor([0])
deletable_row_position  tensor([0])
Here it is :  1
Position of edge to be deleted as columns  tensor([0]) tensor([0, 3, 4])
Deletable row position :  tensor(0)
shape of edge to delete 2
tensor([[ 4., 15.],
        [14., 15.],
        [11., 12.],
        [11.,  9.],
        [12., 13.],
        [ 7.,  8.],
        [ 3.,  4.],
        [ 9.,  2.],
        [13.,  5.],
        [ 8.,  4.],
        [ 1.,  2.],
        [ 1.,  5.]])
thi

The edge to be deleted  tensor([2., 3.])
The shape of the actual edge to be deleted  2
To fix for loop range 2
I should also delete this edge ;  tensor([2., 3.])
Where is it  tensor(3)
go

The new edge is/ is the only  tensor([2., 3.])

Ye
The new point found is at the index  tensor([[3, 1],
        [8, 0]])
 The new point is  3
adjaoined point index  tensor([[6, 0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(6)
go

The new edge is/ is the only  tensor([3., 4.])

Ye
The new point found is at the index  tensor([[6, 1]])
 The new point is  4
adjaoined point index  tensor([[7, 1]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(7)
go

The new edge is/ is the only  tensor([8., 4.])

Ye
The new point found is at the index  tensor([[7, 0]])
 The new point is  8
adjaoined point index  tensor([[5, 1]])
Shape of positions found torch.Size([1, 2])
N

The new edge is/ is the only  tensor([7., 8.])

Ye
The new point found is at the index  tensor([[5, 0]])
 The new point is  7
This forms a loop
traversions :  22
But what was the edge to delete ?  tensor([2., 3.])
lets see
tensor([[10., 11.],
        [ 4., 15.],
        [14., 15.],
        [11., 12.],
        [11.,  9.],
        [12., 13.],
        [ 7.,  8.],
        [ 3.,  4.],
        [ 9.,  2.],
        [13.,  5.],
        [ 8.,  4.],
        [ 1.,  2.],
        [ 1.,  5.]])
adjaoined point index  tensor([[ 9,  1],
        [12,  1]])
Shape of positions found torch.Size([2, 2])
Number of branches rising :  2
The edge to be deleted  tensor([13.,  5.])
The shape of the actual edge to be deleted  2
To fix for loop range 2
I should also delete this edge ;  tensor([13.,  5.])
Where is it  tensor(9)
go

The new edge is/ is the only  tensor([13.,  5.])

Ye
The new point found is at the index  tensor([[ 9,  0],
        [12,  0]])
 The new point is  13
adjaoined point index  tensor([[5, 1]])

...

Where is it  tensor(3)
go

The new edge is/ is the only  tensor([2., 3.])

Ye
The new point found is at the index  tensor([[3, 1],
        [8, 0]])
 The new point is  3
adjaoined point index  tensor([[6, 0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(6)
go

The new edge is/ is the only  tensor([3., 4.])

Ye
The new point found is at the index  tensor([[6, 1]])
 The new point is  4
adjaoined point index  tensor([[7, 1]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(7)
go

The new edge is/ is the only  tensor([8., 4.])

Ye
The new point found is at the index  tensor([[7, 0]])
 The new point is  8
adjaoined point index  tensor([[5, 1]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(5)
go

The new edge is/ is the only  tensor([7., 8.])

Ye
The new point found is at the 

edge_to_delete tensor([ 4., 15.])


What happened :  tensor([ 1,  8, 10]) tensor([ 1,  8, 10])
deletable_row_position  tensor([1])
Here it is :  1
Position of edge to be deleted as columns  tensor([ 1,  8, 10]) tensor([1, 2])
Deletable row position :  tensor(1)
shape of edge to delete 2
tensor([[10., 11.],
        [14., 15.],
        [11., 12.],
        [ 2.,  3.],
        [11.,  9.],
        [12., 13.],
        [ 7.,  8.],
        [ 3.,  4.],
        [ 9.,  2.],
        [ 8.,  4.],
        [ 1.,  2.],
        [ 1.,  5.]])
this shape of position1  torch.Size([1, 2])
the actual position1 is  tensor([[11,  1]])
Where is it  tensor(11)
go

The new edge is/ is the only  tensor([1., 5.])

Ye
The new point found is at the index  tensor([[11,  0]])
 The new point is  1
adjaoined point index  tensor([[10,  0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(10)
go

The new edge is/ is the only  tensor([1., 2.])

Ye
The new

 The new point is  1
adjaoined point index  tensor([[10,  0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(10)
go

The new edge is/ is the only  tensor([1., 2.])

Ye
The new point found is at the index  tensor([[10,  1]])
 The new point is  2
adjaoined point index  tensor([[3, 0],
        [8, 1]])
Shape of positions found torch.Size([2, 2])
Number of branches rising :  2
The edge to be deleted  tensor([2., 3.])
The shape of the actual edge to be deleted  2
To fix for loop range 2
I should also delete this edge ;  tensor([2., 3.])
Where is it  tensor(3)
go

The new edge is/ is the only  tensor([2., 3.])

Ye
The new point found is at the index  tensor([[3, 1],
        [8, 0]])
 The new point is  3
adjaoined point index  tensor([[6, 0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(6)
go

The new edge is/ is the only  tensor([3., 4.])

Ye
The

The new point found is at the index  tensor([[11,  0]])
 The new point is  1
adjaoined point index  tensor([[10,  0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(10)
go

The new edge is/ is the only  tensor([1., 2.])

Ye
The new point found is at the index  tensor([[10,  1]])
 The new point is  2
adjaoined point index  tensor([[3, 0],
        [8, 1]])
Shape of positions found torch.Size([2, 2])
Number of branches rising :  2
The edge to be deleted  tensor([2., 3.])
The shape of the actual edge to be deleted  2
To fix for loop range 2
I should also delete this edge ;  tensor([2., 3.])
Where is it  tensor(3)
go

The new edge is/ is the only  tensor([2., 3.])

Ye
The new point found is at the index  tensor([[3, 1],
        [8, 0]])
 The new point is  3
adjaoined point index  tensor([[6, 0]])
Shape of positions found torch.Size([1, 2])
Number of branches rising :  1
To fix for loop range 1
Where is it  tensor(6)
go

KeyboardInterrupt: 